<a href="https://colab.research.google.com/github/matheus896/My-Blog/blob/main/Crew_Raspagem_Web_(prof%20sandeco).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
!pip install --q crewai
!pip install --q litellm
!pip install --q 'crewai[tools]'
!pip install --q textwrap3

In [135]:
import os
from crewai_tools import ScrapeWebsiteTool
from datetime import datetime
from crewai import Agent, Task, Crew, Process, LLM
from litellm import completion
from textwrap import dedent

# NOSSAS MENTES

In [136]:
os.environ["SAMBANOVA_API_KEY"] = "my_api_key"
LLM = LLM ("sambanova/Meta-Llama-3.1-405B-Instruct",
           temperature=0.2)

# RASPANDO A WEB

In [137]:
ainew = ScrapeWebsiteTool(
    website_url="https://www.artificialintelligence-news.com/"
)

In [138]:
forbes = ScrapeWebsiteTool(
    website_url="https://www.forbes.com/ai/"
)


# Agentes

In [139]:
planejador = Agent(
    role="Planejador de postagem",
    goal="Planejar conteúdo envolvente para instagram sobre {topic}",
    backstory="Você está trabalhando no planejamento de {n} posts para o instagram "
              "sobre o tema: {topic}. "
              "Você coleta informações que ajudam o "
              "público se informar sobre {topic}. "
              "Seu trabalho é a base para que "
              "o Pesquisador de Conteúdo procure na web sobre {topic}.",
    verbose=True,
    llm=LLM,
    allow_delegation=False

)


In [140]:
pesquisador = Agent(
    role='Pesquisador',
    goal='Pesquisar tendências para postagens sobre {topic} na área '
         'de tecnologia com base no planejamento do Planejador. '
         'Seu trabalho é a base para que '
         'o escritor possa escrever {n} posts sobre {topic}',
    verbose=True,
    backstory="Você é um pesquisador experiente, sempre em busca das últimas tendências e informações relevantes sobre {topic}.",
    llm=LLM,
    tools=[ainew, forbes],
    allow_delegation=False
)

In [141]:
escritor = Agent(
    role='Escritor',
    goal='Escrever {n} postagens cativantes em português do Brasil para o Instagram sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras.'
         'Seu trabalho é a base para que o fotografo possa escrever prompts de imagens para os {n} posts',
    backstory="Você é um escritor criativo, capaz de transformar informações em conteúdo atraente para postagens no Instagram.",
    llm=LLM,
    verbose=True,
    allow_delegation=False
)


In [142]:

gerente = Agent(
    role='Gerente de postagens',
    goal=dedent("""Supervisione o trabalho de uma equipe de postagens no Instagram. Você é bem crítico em relação
            ao que vai ser postado no Instagram da empresa de notícias na área da tecnologia.
            Você delegará tarefas à sua equipe e fará perguntas esclarecedoras
            para revisar e aprovar as {n} posts sobre {topic} que foram solicitadas pela direção da empresa."""),
    backstory=dedent("""Você é um gerente experiente, sempre em busca das últimas tendências e informações relevantes.
                 Você está trabalhando com uma nova demanda e faz com que sua equipe realize o trabalho da
                 melhor forma possível."""),
    llm=LLM,
    verbose=True

)


# Tarefas

In [144]:
plano_task = Task(
    description=(
        "1. Priorize as últimas tendências, principais 'players', "
            "e notícias relevantes sobre {topic}.\n"
        "2. Identifique o público-alvo, considerando "
            "seus interesses e pontos de dor.\n"
        "3. Desenvolva um plano de conteúdo detalhado, incluindo "
            "uma introdução, pontos principais e um chamado à ação.\n"
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento de plano de conteúdo para {n} posts sobre {topic} "
        "com um esboço, análise do público, "
        "palavras-chave de SEO e recursos.",
    agent=planejador,
    verbose=True
)

In [145]:
pesquisa_task = Task(
    description="Pesquise as últimas tendências sobre {topic}.",
    expected_output="Um relatório detalhado sobre as tendências mais recentes sobre {topic} na área de tecnologia.",
    agent=pesquisador,
    verbose=True
)

In [146]:
escrita_task = Task(
    description=dedent("""Escreva {n} postagens envolventes em português do Brasil
                        com base nas tendências pesquisadas sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras cada.
                        Cada post deve ser formatado como:
                        \n\nPOST:\ntexto do post em português do brasil
                        \n\nPROMPT:\nPrompt da imagem desse post\n\n"""),
    expected_output="{n} postagens de Instagram bem escritas, atraentes e em português do Brasil, formatadas conforme especificado para o tópico {topic}.",
    agent=escritor,
    verbose=True
)


In [147]:
revisao_task = Task(
description=dedent("""Revise as {n} escritas para as {n} postagens do cliente e garanta
                          que as informações de cada postagem estejam organizadas, sem erros e cativantes
                          em português do Brasil sobre {topic}.
                          Certifique-se de que cada post está formatado como:
                          \n\nPOST:\ntexto do post em português do brasil
                          \n"""),
expected_output="{n} textos organizados por post, revisados e prontos para serem publicados em português do Brasil, formatados conforme especificado.",
agent=gerente,
verbose=True

)



# CRIANDO A TRIPULAÇÃO

In [148]:
crew = Crew(
    agents=[planejador, pesquisador, escritor, gerente],
    tasks=[plano_task, pesquisa_task, escrita_task, revisao_task],
    process=Process.sequential,
    verbose=True,
    memory=True
)


In [149]:
inputs={
      'topic':'Inteligência Artificial e Agentes Inteligentes',
      'n':1}

In [152]:
result = crew.kickoff()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: fake. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [150]:

current_date = datetime.now().strftime("%Y-%m-%d")
filename = f"posts-{current_date}.txt"              #"post-2024-06-11.txt"
with open(filename, 'w', encoding='utf-8') as file:
    file.write(result)
